In [1]:
from koselleck import *

In [2]:
swas=get_word_abstractness_scores()

In [3]:
def prdz(y,ystart=1710,yend=3000,ystep=40):
    ln=None
    for n in range(ystart,yend,ystep):
        if ln is None: ln=n
        if y<n: return ln
        ln=n
        
prdz(1715)

1710

In [4]:
df=pd.read_pickle('data/data.all_neighborhoods.v2.pkl').reset_index()
df['period_int']=df.period.apply(lambda x:int(x[:4]))



df['period2']=df.period_int.apply(prdz)
# df['period2']=df['period_int']
df

,period,run,word,neighborhood,period_int,period2
0,1700-1710,run_01,improve,"discern, promote, amend, recommend, assume, ac...",1700,1710
1,1700-1710,run_01,mechanical,"chymistry, philosophy, sublime, theory, theolo...",1700,1710
2,1700-1710,run_01,reform,"foreseen, blasphemous, owned, incur, lawfully,...",1700,1710
3,1700-1710,run_01,administration,"dominion, rulers, judicature, episcopacy, syno...",1700,1710
4,1700-1710,run_01,society,"member, societies, teachers, proposals, unifor...",1700,1710
...,...,...,...,...,...,...
81445,1890-1900,run_25,feudal,"feudalism, stuarts, patriarchal, tribal, plant...",1890,1870
81446,1890-1900,run_25,unconscious,"conscious, unconsciously, oblivious, insensibl...",1890,1870
81447,1890-1900,run_25,medieval,"mediaeval, ancient, byzantine, gothic, modern,...",1890,1870
81448,1890-1900,run_25,dramatic,"drama, lyrical, poetic, shakespearean, lyric, ...",1890,1870


In [5]:
df.groupby('period2').size()

period2
1710    14700
1750    15250
1790    16775
1830    19300
1870    15425
dtype: int64

In [6]:
dfabs=get_dfpiv_abs().reset_index().melt(id_vars=['word'],value_name='abs')
dfabs['period2']=dfabs['period'].apply(prdz)
dfabs=dfabs.groupby(['word','period2']).mean().reset_index().dropna()
okwords=set(dfabs.word)
dfabs

,word,period2,abs
1,abandonment,1750,1.327681
2,abandonment,1790,2.285290
3,abandonment,1830,2.761914
4,abandonment,1870,2.678389
5,abatement,1710,0.316051
...,...,...,...
29999,zoo,1870,-0.784011
30001,zoology,1750,0.320236
30002,zoology,1790,0.013060
30003,zoology,1830,0.073992


In [7]:
def synth(df,max_rank=25):
    o=[]
    for (w,p),wdf in df.groupby(['word','period2']):
        counts=Counter()
        ranks=defaultdict(list)
        for neighb in wdf.neighborhood:
            for i,w2 in enumerate(neighb.split(', ')):
#                 if max_rank and i>=max_rank: break
                ranks[w2]+=[i+1]
                counts[w2]+=1
        ow=[]
        for w2 in ranks:
            odx={
                'word':w,
                'period':p,
                'word2':w2,
                'rank':np.mean(ranks[w2]),
                'count':counts[w2]
            }
            ow.append(odx)
        owdf=pd.DataFrame(ow)
        owdf['rankcount']=owdf['count'] * (1/ (owdf['rank']+1))
        owdf['rank2']=(-owdf['rankcount']).rank()
        o.append(owdf)
    return pd.concat(o)


In [8]:
dfres=synth(df).set_index('word')
dfres

,period,word2,rank,count,rankcount,rank2
word,,,,,,
administration,1710,dominion,11.181818,22,1.805970,34.0
administration,1710,rulers,21.727273,11,0.484000,139.0
administration,1710,judicature,19.647059,17,0.823362,87.0
administration,1710,episcopacy,12.750000,24,1.745455,35.0
administration,1710,synodical,7.600000,25,2.906977,15.0
...,...,...,...,...,...,...
world,1870,illusions,47.000000,1,0.020833,221.0
world,1870,monthlies,46.000000,1,0.021277,214.5
world,1870,era,33.000000,1,0.029412,180.0


In [9]:
w='culture'
max_rank=25
wdf=dfres.loc[w]

In [10]:
# wdfp=wdf.pivot('word2','period','rank2').fillna(max_rank).applymap(lambda x: x if x<max_rank else max_rank)
# wdfp

In [11]:
# wdfp.reset_index().melt(id_vars=['word2'],value_name='rank2')

In [12]:
words_ever_high=set(wdf.query('rank<10').word2)
len(words_ever_high)

23

In [13]:
dfres.groupby(['period','word','word2']).mean()

rank  count  rankcount  rank2
period word           word2                                         
1710   administration abhorrence  29.541667     24   0.785812   93.0
                      abilities   46.000000      1   0.021277  626.0
                      ability     24.500000      4   0.156863  270.0
                      abolished   42.000000      1   0.023256  598.0
                      absolute    30.468750     32   1.016882   76.0
...                                     ...    ...        ...    ...
1870   world          womankind   30.437500     16   0.508946   62.0
                      workaday    31.428571      7   0.215859   88.0
                      world's      1.666667     75  28.125000    1.0
                      worlds       3.533333     75  16.544118    3.0
                      yearnings   38.562500     16   0.404423   67.0

[344662 rows x 4 columns]

In [23]:
def show(w='culture',min_periods=2,max_rank=20,max_rank1=15,width=8,height=6,
        save=False,vnum='v4'):
    if w in set(dfres.index):
        wdf=dfres.loc[w]
        wdf=wdf[wdf.word2.isin(okwords)]
        wdf=wdf[~wdf.word2.isin({'foil'})]
        figdf=wdf
        words_ever_high=set(wdf[wdf.rank2<=max_rank1].word2)
# #         print(words_ever_high)
#         figdf=figdf[figdf.rank2<max_rank*4]
#         s=figdf[figdf.rank2<max_rank*4].word2.value_counts()
# #         display(wdff[wdff.word2=='soils'])
# #         print(s)
#         badwords=set(s[s<min_periods].index)
# #         print(badwords)
#         figdf=figdf[~figdf.word2.isin(badwords)]
        figdf=figdf[figdf.word2.isin(words_ever_high)]
        wdfp=figdf.pivot('word2','period','rankcount')#.fillna(0).applymap(lambda x: x if x<max_rank else max_rank)
        figdf=wdfp.reset_index().melt(id_vars=['word2'],value_name='rankcount')
        
        s=figdf.word2.value_counts()
        badwords=set(s[s<min_periods].index)
        figdf=figdf[~figdf.word2.isin(badwords)]
        
        
        figdf=pd.concat(grp.assign(rank3=(-1*grp['rankcount']).rank(method='first')) for i,grp in figdf.groupby('period'))
#         return figdf
#         return figdf
        figdf=figdf[figdf.rank3<max_rank]
        figdf=figdf.sort_values('rankcount')#.set_index('word2')
        
        
        
        
#         figdf['abs']=-1 * swas
#         figdf['conc']=swas
        
        figdf=figdf.merge(dfabs, left_on=['word2','period'], right_on=['word','period2'])
        figdf['conc']=figdf['abs']*-1
        figdf['is_abs']=figdf['abs']>0
#         figdf=figdf.reset_index()
#         display(figdf)
        fig=start_fig(
            figdf,
            x='period',
            y='rank3',
            label='word',
            group='word',
            fill='conc',
            figure_size=(width,height)
        ) + p9.geom_line(size=0.5) + p9.geom_label(size=7) + p9.scale_y_reverse()#limits=[0,max_rank])
#         fig+=p9.scale_color_gray()
#         fig+=p9.scale_fill_gray()
#         fig+=p9.scale_fill_distiller(type='div',direction=1)
        fig+=p9.scale_fill_gradient(low='white',high='#666666')
        fig+=p9.labs(
            title=f'Changing associations of "{w}"',
            x='Twenty-year period (BPO)',
            y=f'Proximity to "{w}" →',
            fill='Concreteness'
        )
#         fig+=p9.scale_fill_manual({True:'silver',False:'gray'})
        if save:
            ofn=f'fig.changing_simple.{w}.{vnum}.png'
            fig.save(f'figures/{ofn}')
            fig.save(f'/home/ryan/Markdown/Drafts/TheGreatAbstraction/figures/{ofn}')
        return fig
    else:
        print(f'!! {w} not in data')
    
    
res=interact_manual(show, w='demand')

interactive(children=(Text(value='demand', description='w'), IntSlider(value=2, description='min_periods', max…